<img src="cats-eye.png" align="center"/>

# Lab 2B: Advection-Diffusion equation (2D)

This lab extends some of the ideas we saw in the previous lab to two dimensions, and examine the role of the velocity field in advecting the dye concentration field. 

To use this notebook, you will need Python 3 and the latest version of [Dedalus](http://dedalus-project.org/) installed on your local machine. 

# Background

## The Childress-Soward cat's-eye flow

In this experiment we will use a periodic 2D velocity field described by the one-parameter family of streamfunctions

$$
\psi(x,y) = \sin x \, \sin y + \epsilon \, \cos x \, \cos y.
$$

The streamfunction $\psi(x,y)$ describes the path taken by particles advected by the (time-independent) flow. Constant values of $\psi(x,y)$ are the _streamlines_ of the flow. The velocities in the $x$ and $y$ direction are calculated from the derivatives of the streamfunction: 

$$
U(x,y) = - \partial_y \, \psi = - \sin x \, \cos y + \epsilon \, \cos x \, \sin y,
$$

$$
V(x,y) = \partial _x \, \psi = \cos x \, \sin y - \epsilon \, \sin x \, \cos y.
$$

The parameter $\epsilon$ controls the shape of the flow: for $\epsilon = 0$, the flow forms a periodic array of closed cells, for $\epsilon = 1$, the flow is a parallel shear flow, and for values of $0 < \epsilon < 1$, the flow forms elongated "cat's eye" ellipses. 

The next cell plots the streamlines for different values of the parameter $\epsilon$. 

In [ ]:
# inmport libraries
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# plot the flow field
n = 128
x,y = np.linspace(-np.pi,np.pi,n)[1:-1, None], np.linspace(-np.pi,np.pi,n)[None,1:-1]
xf,yf = x.flatten(),y.flatten()
xx,yy = np.meshgrid(xf,yf,indexing='ij')

f = plt.figure(figsize=(16,4))
ax = f.add_subplot(1,4,1)
ax.contour(xx, yy, np.sin(xx)*np.sin(yy) + 0.0*np.cos(xx)*np.cos(yy))
ax.set(aspect=1,title='eps = 0.0')

ax = f.add_subplot(1,4,2)
ax.contour(xx, yy, np.sin(xx)*np.sin(yy) + 0.1*np.cos(xx)*np.cos(yy))
ax.set(aspect=1,title='eps = 0.1')

ax = f.add_subplot(1,4,3)
ax.contour(xx, yy, np.sin(xx)*np.sin(yy) + 0.5*np.cos(xx)*np.cos(yy))
ax.set(aspect=1,title='eps = 0.5')

ax = f.add_subplot(1,4,4)
ax.contour(xx, yy, np.sin(xx)*np.sin(yy) + 1.0*np.cos(xx)*np.cos(yy))
ax.set(aspect=1,title='eps = 1.0')

## Advection-diffusion equation

Using the above flow field, we will evolve a concentration field $C(x,y,t)$ with the 2D advection diffusion equation

$$
\partial_t \, C + U(x,y) \, \partial_x \, C + V(x,y) \, \partial_y \, C = K \, \left( \partial_x^2 \, C + \partial_y^2 \, C \right).
$$

To write this a form suitable for Dedalus, we will introduce two new fields, $C_x$ and $C_y$, so that the advection-diffusion equation becomes

$$
\partial_t \, C - K \, \partial_x \, C_x - K \, \partial_y \, C_y = - U(x,y) \, C_x - V(x,y) \, C_y.
$$

Notice that we have rearranged the PDE so that all the linear terms are on the LHS and everything else is on the RHS. 

In addition we have two new PDEs that come from the definitions of $C_x$ and $C_y$

$$
C_x - \partial_x \, C = 0 \quad \mbox{and} \quad C_y - \partial_y \, C_y = 0.
$$

Finally, we impose periodic boundary conditions in $x$ and $y$. Thus, any dye that moves off the right side of the domain will re-enter from the left side, for example. 

### References

- S. Childress and A. M. Soward, “Scalar transport and alpha-effect for a family of cat’s-eye flows,” J. Fluid Mech. 205, 99 (1989).

# Experimental setup

## Dedalus

Import the `dedalus` library, some extra tools (`flow_tools`) for Dedalus, and a library called `time` that will tell us how much time has elapsed since we started the calculation. Finally, suppress some logging messages. 

In [ ]:
from dedalus import public as de
from dedalus.extras import flow_tools
import time

import logging
logger = logging.getLogger(__name__)

## Bases and Domains

The domain used in this problem is periodic in $x$ and $y$. Thus, we will choose sines and cosines (Fourier) for both the $x$ and $y$ basis functions. The domain is then constructed by putting these basis functions together. 

In [ ]:
# Parameters
L = np.pi # 2*L = length of domain
nx = 64   # number of gridpoints
K = 0.01    # diffusivity
eps = 0.  # controls shape of velocity field 

# Create bases and domain
x_basis = de.Fourier('x', nx, interval=(-L,L))
y_basis = de.Fourier('y', nx, interval=(-L,L))
domain = de.Domain([x_basis,y_basis], grid_dtype=np.float64)

## Initial value problem

Again, this is an initial value problem, which we call using the command IVP from the Dedalus library. We also need to tell Dedalus the domain ($x$, $y$), the dynamical variables ($C$, $C_x$, $C_y$), parameters ($K$, $\epsilon$) and functions ($U(x,y)$, $V(x,y)$) that appear in the problem. 

In [ ]:
# Formulate the initial value problem
problem = de.IVP(domain, variables=['C','Cx','Cy'])

# Set parameters (diffusivity)
problem.parameters['K'] = K
problem.parameters['eps'] = eps

# Set velocity field
problem.substitutions['U'] = '-sin(x)*cos(y) + eps*cos(x)*sin(y)'
problem.substitutions['V'] = ' cos(x)*sin(y) - eps*sin(x)*cos(y)'

## Formulating the problem

Now we add the equations. Notice that there are no explicit boundary conditions, since these are already satisfied by the periodic basis functions chosen for $x$ and $y$. 

In [ ]:
problem.add_equation("dt(C) - K*dx(Cx) - K*dy(Cy) = -U*Cx -V*Cy")
problem.add_equation("Cx - dx(C) = 0")
problem.add_equation("Cy - dy(C) = 0")

## Building a solver

Now we build the solver and specify the stop criteria. Let's stop after the model time reaches 10, or the solver takes 30 minutes. 

In [ ]:
# Build solver
solver = problem.build_solver(de.timesteppers.RK222)
logger.info('Solver built')

# timesteps
T  = 10
dt = 1/500

# Integration parameters
solver.stop_sim_time = T
solver.stop_wall_time = 30 * 60.
solver.stop_iteration = np.inf

## Setting the initial condition

We'll use the same initial condition as was used in the 1D problem. The only difference is that we are working in 2D now, so we will make initial condition isotropic about the center of the doman. 

In [ ]:
# Get the bases from the object "domain" and the state variables from the object "solver"
x, y = domain.grid(0), domain.grid(1)
C = solver.state['C']
Cx = solver.state['Cx']
Cy = solver.state['Cy']

n = 20
C['g'] = np.log(1 + np.cosh(n)**2/np.cosh(n*np.sqrt(x**2+y**2))**2) / (2*n)
C.differentiate(0, out=Cx)
C.differentiate(1, out=Cy)

C.set_scales(1, keep_data=True)
xx,yy = np.meshgrid(x,y,indexing='ij')
f = plt.figure(figsize=(8,8))
ax = f.add_subplot(1,1,1)
ax.pcolormesh(xx, yy, C['g'])
ax.set(aspect=1,title='C (t = 0)',xlabel='x',ylabel='y')

## Solving the problem

Now we are ready to solve the problem. First we need to save some data for the final plots. Then we can run the main time loop. 

In [ ]:
# Store data for final plot
C.set_scales(1, keep_data=True)
C_list = [np.copy(C['g'])]
t_list = [solver.sim_time]

In [ ]:
# Main loop
while solver.ok:
    solver.step(dt)
    if solver.iteration % 20 == 0:
        C.set_scales(1, keep_data=True)
        C_list.append(np.copy(C['g']))
        t_list.append(solver.sim_time)
    if solver.iteration % 100 == 0:
        logger.info('Iteration: %i, Time: %e, dt: %e' %(solver.iteration, solver.sim_time, dt))

In [ ]:
# Make plot of C
f = plt.figure(figsize=(16,4))
print(len(C_list))

for i in range(4):
    ax = f.add_subplot(1,4,i+1)
    ax.pcolormesh(xx, yy, C_list[i*80])
    ax.set(aspect=1)

In [ ]:
# plot average of C as a function of time
Cavg_list = np.sum(np.sum(C_list[0]))/nx**2

for i in range(len(C_list)):
    Cavg_list = np.append(Cavg_list,np.sum(np.sum(C_list[i]))/nx**2)
    
# Make plot of C average
t_plot = np.linspace(0,T,len(Cavg_list))
plt.plot(t_plot,Cavg_list)
plt.ylim(0,.1)

In [ ]:
# plot variance of C as a function of time
Cvar_list = np.sum(np.sum(C_list[0]**2))

for i in range(len(C_list)):
    Cvar_list = np.append(Cvar_list,np.sum(np.sum(C_list[i]**2)))
    
# Make plot of C
t_plot = np.linspace(0,T,len(Cvar_list))
plt.plot(t_plot,Cvar_list)

## Now try it yourself

Repeat the experiment with different values of `K` and `eps`. Do you notice a difference in the rate of decay of the concentration? Which is more efficienty at diffusing the dye: closed cells, parallel shear flow, or something in between? 